In [5]:
%matplotlib qt
#%matplotlib inline
import os
import csv
import numpy as np
import re
import datetime
import fnmatch
from pathlib import Path
from itertools import groupby
import copy
import itertools
import distance
import nltk
#nltk.download('stopwords')
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from nltk.stem import SnowballStemmer
from nltk.stem import snowball

from itertools import *
from operator import *

from openpyxl import load_workbook

In [6]:
# exceptional removal of particular extra sentences not typed by the user 
dict_phraseStim = {
    #'2019-02-05-14-10-39_2ndPart_2' : [1, 2, 3, 4, 5, 6, 9, 10],
    #'2019-01-14-14-58-30' : [0], # ys, session_trial ()
    '2019-01-16-16-36-17_1stPart_2' : [-1], # af_session1
    '2019-01-16-17-00-12_2ndPart_2': [1], # af_session1
    '2019-01-17-15-27-20_1stPart_2' : [4], # Af session2
    '2019-01-17-16-03-27_2ndPart_2' : [0, 1, 2], # Af session2
    '2019-02-06-11-25-41_1' : [7],               # aq_session1    
    '2019-02-08-11-33-53_1stPart_1' : [1],  # aq session3_1_part1
    '2019-02-08-12-11-34_2ndPart_1' : [0, 1, 2, 3],  # aq session3_1_part2
    '2019-01-31-09-37-5_2ndPart_2' : range(1,5), # bh1, session 4 , all sentences except the first one deleted
    '2019-01-31-09-22-49_1stPart_2' : [4],  # bh1_session4_2_part1
    '2019-02-21-16-09-44_1stPart_1' : [1], # bh2_session1
    '2019-02-21-16-22-22_2ndPart_1' : [2, 3, 4],# bh2_session1
    '2019-02-28-17-03-53_1stPart_2' : [2],       # bh2_session3
    '2019-02-28-17-24-2_2ndPart_2' : [0, 2],     # bh2_session3
    '2019-02-14-13-28-20_1stPart_2' : [2], # cw_session3_2_part1
    '2019-02-14-13-57-41_2ndPart_2' : [0, 2, 3], # cw_session3_2_part2
    '2019-02-21-15-01-4_1stPart_1' : [0],        # le_session3
    '2019-02-21-15-25-56_2ndPart_1' : [1],        # le_session3
    #'2019-02-18-10-28-35_2' : [0],               # ls2_session4 # picture not described
    '2019-02-05-14-00-27_1stPart_2' : [3],        # mh_session1
    '2019-02-05-14-10-39_2ndPart_2' : [0, 1, 3],   # mh_session1
    '2019-02-08-10-51-3_1stPart_1' : [4],        # mn_session1
    '2019-02-08-11-05-7_2ndPart_1' : [0, 2, 3, 4], # mn_session1
    '2019-02-19-10-34-7_1stPart_1' : [3],          # mn_session3
    '2019-02-19-10-56-43_2ndPart_1' : [1, 2, 3, 4], # mn_session3
    '2019-01-16-15-18-0_1' : [4],            # no_session1
    '2019-02-19-17-10-45_1' : [3],                  # ph_session5
    '2019-01-29-13-25-4_1' : [3],        # ph_session2
    '2019-03-07-16-44-5_2' : [1],                   # rh_session1
    '2019-03-14-13-56-56_2' : [2],                  # rh_session3
    '2019-01-14-15-07-21_1' : [4], # ys_session1
    '2019-01-16-15-18-50_1stPart_1' : [3, 4], # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [2], # ys_session2
    '2019-01-30-11-22-25_1' : [3, 5, 7],          # ys_session4
    '2019-01-30-11-22-25_1' : [4, 6, 7] # ys, session 4
}

# exceptional removal of sentences/words typed by the user, but then deleted everything to have a blank scratchpad

dict_phraseUser = {
    "2019-02-06-15-44-15_1" : [2, 3, 6], 
    "2019-02-06-16-19-9_2" : [1, 3, 6, 7],
    "2019-02-12-11-21-21_2" : [0],
    "2019-02-14-14-28-49_1" : [0, 2, 3], # ac_session3_1
    "2019-02-14-14-45-49_2" : [0, 5, 6], # ac_session3_2
    '2019-01-29-14-19-26_1' : [0, 3, 4], # bh1_session2_1
    '2019-01-29-14-40-36_2' : [0, 1, 2], # bh1_session2_2
    '2019-01-30-14-29-29_2' : [4],       # bh1_session3_2
    '2019-01-31-09-12-2_1' : [3],         # bh1_session4_1
    '2019-01-31-09-22-49_1stPart_2' : [4], # bh1_session4_2_part1
    '2019-03-05-09-15-11_1' : [1],         # bh2_session5_1
    '2019-03-05-09-15-11_2' : [1],        # bh2_session5_2
    '2019-02-21-15-55-56_2' : [2],       # ch_session5_2
    '2019-01-30-15-19-36_2' : [1],       # jm_session2_1
    '2019-01-30-15-04-30_1' : [0],         # jm_session2_2
    '2019-01-16-15-18-50_1stPart_1' : [1],  # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [0], # ys_session2
    '2019-01-30-11-22-25_1' : [2, 4],       # ys_session4
    '2019-01-30-11-57-3_2' : [0] ,          # ys_session4
    '2019-01-31-13-13-2_1' : [4],           # ys_session5
    '2019-01-30-10-20-32_1' : [0, 1, 2, 3, 4, 5], # no_session4
    '2019-01-30-10-46-38_2' : [0],          # 
    '2019-02-28-17-03-53_1stPart_2' : [2],   # bh2_session3
    '2019-03-12-09-30-5_1' : [0],            # kj_session3
    '2019-02-13-15-20-38_1' : [0, 1, 2, 3, 6], # ls1_session3
    '2019-02-18-10-25-52_1' : [1],              # ls2_session4
    '2019-02-18-10-46-26_2' : [0],            # ls2_session4
    '2019-01-29-13-25-4_1' : [0, 1, 7],        # ph_session2
    '2019-01-29-13-43-50_2' : [0],              # ph_session2
    '2019-03-07-16-17-30_1' : [0],              # rh_session1
    '2019-03-07-16-44-5_2' : [0, 1],         # rh_session1
    '2019-03-14-13-56-56_2' : [0, 1, 3]         # rh_session3
}

# key selection can have extra selections of NextPhrase at the end
dict_keySelectionOfNextPhrase = {
    "2019-02-11-11-18-30_1" : [12, 13], # ac_session1
    "2019-01-16-17-00-12_2ndPart_2" : [12], # af_session1
    "2019-01-17-15-27-20_1stPart_2" : [12], # af_session2
    "2019-02-06-16-19-9_2" : [12], # af_session3
    "2019-02-12-11-07-43_1" : [12], # af_session4
    "2019-02-27-15-08-32_1" : [12], # af_session5
    "2019-01-28-14-30-44_1" : [12], # bh1_session1
    "2019-02-21-16-22-22_2ndPart_1" : [12], # bh2_session1
    "2019-02-18-14-02-56_2" : [12], # le_session1
    "2019-02-19-10-03-14_1" : [12], # le_session2
    "2019-02-08-11-05-7_2ndPart_1" : [12], # mn_session1
    "2019-02-08-11-12-51_2" : [12, 13], # mn_session1
    "2019-02-15-11-38-22_1" : [12, 13], # mn_session2
    "2019-02-15-11-54-25_2" : [12], # mn_session2
    "2019-01-16-15-18-0_1" : [12], # no_session1
    "2019-01-28-13-31-51_1" : [12], # ph_session1
    "2019-01-28-13-49-14_2" : [12], # ph_session1
    "2019-01-14-15-07-21_1" : [12], # ys_session1
    "2019-01-17-15-05-1_1" : [12], # ys_session3
    "2019-01-30-11-22-25_1" : [12], # ys_session4
    "2019-01-31-13-32-2_2" : [12], # ys_session5
}


# key selection when participants skips some sentences
dict_keySelectionNotCompleted = {
    "2019-01-16-16-36-17_1stPart_2" : [0, 1, 3, 5, 7], # af_session1 ---- last sentence is not finished
    "2019-01-16-17-00-12_2ndPart_2" : [0, 1, 3, 4, 5, 7, 9, 11], # af_session1
    "2019-01-17-15-27-20_1stPart_2" : [0, 1, 3, 5, 7, 9, 11], # af_session2 
    "2019-01-17-16-03-27_2ndPart_2" : [0, 1, 2, 3, 4, 5, 6, 7, 9, 11], # af_session2
    "2019-02-08-11-33-53_1stPart_1" : [0, 1, 3, 4, 5, 7, 9, 11], # aq_session3
    "2019-02-08-12-11-34_2ndPart_1" : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11], # aq_session3
    "2019-01-28-14-30-44_1" : [0, 1, 3, 5], # bh1_session1
    "2019-01-31-09-22-49_1stPart_2": [0, 1, 3, 5, 7, 9, 10, 11], # bh1_session4
    "2019-01-31-09-37-5_2ndPart_2" : [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11], # bh1_session4
    "2019-02-21-16-09-44_1stPart_1" : [0, 1, 3, 4, 5, 7, 9, 11], # bh2_session1
    "2019-02-21-16-22-22_2ndPart_1" : [0, 1, 3, 5, 6, 7, 8, 9, 10, 11], # bh2_session1
    "2019-02-28-17-03-53_1stPart_2" : [0, 1, 3, 5, 6, 7, 9, 11], # bh2_session3
    "2019-02-28-17-24-2_2ndPart_2" : [0, 1, 2, 3, 5], # bh2_session3     ----
    "2019-02-14-13-28-20_1stPart_2" : [0, 1, 3, 5, 6, 7, 9, 11], # cw_session3
    "2019-02-14-13-57-41_2ndPart_2" : [0, 1, 2, 3, 5, 6, 7, 8, 9, 11], # cw_session3
    "2019-02-21-15-01-4_1stPart_1" : [0, 1, 2, 3, 5, 7, 9, 11], # le_session3
    "2019-02-21-15-25-56_2ndPart_1" : [0, 1, 3], # le_session3       ----
    "2019-02-05-14-00-27_1stPart_2" : [0, 1, 3, 5, 7, 8], # mh_session1
    "2019-02-05-14-10-39_2ndPart_2" : [0, 1, 2, 3, 4, 5, 7, 8, 9, 11], # mh_session1
    "2019-02-08-10-51-3_1stPart_1" : [0, 1, 3, 5, 7, 9, 10, 11], # mn_session1
    "2019-02-08-11-05-7_2ndPart_1" : [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11], # mn_session1
    "2019-02-19-10-34-7_1stPart_1" : [0, 1, 3, 5, 7, 8, 9, 11], # mn_session3
    "2019-02-19-10-56-43_2ndPart_1" : [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11], # mn_session3
    "2019-01-29-13-25-4_1" : [0, 1, 3, 5, 7], # ph_session2  -- sessions where there are less score questions 
    # and more sentences typed
    "2019-01-16-15-18-50_1stPart_1" : [0, 1, 3, 5, 7, 8, 9, 10], # ys_session2
    "2019-01-17-15-05-1_1" : [0, 1, 3, 5],  # ys_session3  -- sessions where there are less score questions 
    # and more sentences typed
    "2019-02-06-11-25-41_1" : [0, 1, 3, 5, 11], # aq_session1 -- sessions where there are less score questions 
    # and more sentences typed
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 2, 5], # ys_session2 -- different for reading and writing, this one is for
    # writing
    '2019-01-30-11-22-25_1' : [0, 1, 3, 5, 7, 9, 11]   # ys_session4 -- sessions where there are less score questions 
    # and more sentences typed
   
}

# dictionary for phrase removal just like in the dict_phraseStim, but since not all participants require that, some that 
# do, are added to this new dictionary here
dict_keySelection_phraseStim = {
    '2019-01-17-15-27-20_1stPart_2' : [4], # Af session2
    '2019-01-16-15-18-0_1' : [4],        # no_session1
    '2019-02-19-17-10-45_1' : [3],                  # ph_session5
    '2019-03-07-16-44-5_2' : [1],        # rh_session1
    '2019-03-14-13-56-56_2' : [2],              # rh_session3
    '2019-01-14-15-07-21_1' : [4]         # ys_session1
}


# in the beginning experiments, not everyone started with 800 initial dwell time

dict_dwellTimeOrig_not800 = {
    "2019-01-16-15-51-13_2" : 600, # no_session1
    "2019-01-16-15-18-0_1" : 600, # no_session1
    "2019-01-16-15-43-8_1" : 100, # af_session1
    "2019-01-16-16-36-17_1stPart_2" : 100, # af_session1
    "2019-01-16-17-00-12_2ndPart_2" : 100, # af_session1
    "2019-01-17-15-03-40_1" : 100, # af_session2
    "2019-01-17-15-27-20_1stPart_2" : 0, # af_session2
    "2019-01-17-16-03-27_2ndPart_2" : 100, # af_session2
    "2019-01-14-15-07-21_1" : 500, # ys_session1
    "2019-01-14-15-25-55_2" : 300, # ys_session1
    "2019-01-16-15-18-50_1stpart_1" : 200, # ys_session2
    "2019-01-16-15-42-51_2ndPart_1" : 100, # ys_session2
    "2019-01-16-15-59-55_2" : 100, # ys_session2
    "2019-01-17-15-05-1_1" : 100, # ys_session3
    "2019-01-17-15-31-12_2" : 100 # ys_session3
}


# list of all things that should be present when computing effective time
list_keysToBeCounted = ['Comma', 'BackOne', 'BackMany', 'SpaceBar']

# some sessions do not have gaze data
dict_noGazeData = {
    '2019-01-16-17-00-12_2ndPart_2' : 'no gaze data', # af_session2
    '2019-01-17-15-31-12_2' : 'no gaze data', #ys_session2
    '2019-01-30-11-57-3_2' : 'no gaze data' # ys_session4
}



In [7]:
dict_keySelection_ReadingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 3, 5], # ys_session2 
}

dict_keySelection_WritingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 2, 5], # ys_session2   
}

# normally, reading part of trial ends when people look at the keyboardwithphrases. For some trials, this is not done,
# as the reading is done, and the trial is accidentally skipped, and written in the next trial. Here, the trial number 
# given will have the reading time ending as sleep, and not keyboard with phrases. 
dict_keyboardNotChange_ReadingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : 0, # ys_session2 
}

dict_keySelection_firstSleepNotCounted = {
    "2019-01-28-14-50-41_2" : (0, 2), # bh1_session1 -- 3rd sleep activation to be counted
    "2019-02-19-10-56-43_2ndPart_1" : 2  # mn_session3 -- 3rd sleep activation is to be counted
}

In [8]:
TimeDwellOrig = 800
TimeFixation = 300

In [9]:
def FixUserKeys(UserKeys_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    Column_beforeDecimal = [item[2] for item in UserKeys_Old]
    Column_afterDecimal = [item[3] if len(item)>3 else '00' for item in UserKeys_Old]
    
    UserKeys_ProgressPercent = [float(Column_beforeDecimal[i]+'.'+ Column_afterDecimal[i]) for i in 
                                range(0, len(Column_beforeDecimal))]
    UserKeys_Times = [item[0] for item in UserKeys_Old]
    UserKeys_Keys = [item[1] for item in UserKeys_Old]
    
    UserKeys_New = [[UserKeys_Times[ind], UserKeys_Keys[ind], UserKeys_ProgressPercent[ind]] for ind in 
                    range(0, len(UserKeys_ProgressPercent))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    
    return UserKeys_New
        

In [10]:
def FixScratchPad(ScratchPad_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    ScratchPad_Times = [item[0] for item in ScratchPad_Old]
    
    ScratchPad_Phrases = list()
    
    # loop to combine phrases divided by commas
    ScratchPadInd = -1 
    while ScratchPadInd < len(ScratchPad_Old)-1:
        ScratchPadInd = ScratchPadInd + 1
        commasInPhrase = len(ScratchPad_Old[ScratchPadInd])-2
        if commasInPhrase < 1:
            #print(ScratchPad_Old[ScratchPadInd][1])
            ScratchPad_Phrases.append(ScratchPad_Old[ScratchPadInd][1])
            continue
        scratchPadPhrase = ScratchPad_Old[ScratchPadInd][1]
        for phraseJoinNr in range(1, commasInPhrase+1):
            scratchPadPhrase = scratchPadPhrase + ', ' + ScratchPad_Old[ScratchPadInd][1+phraseJoinNr]
        
        ScratchPad_Phrases.append(scratchPadPhrase)
            
        
    ScratchPad_New = [[ScratchPad_Times[ind], ScratchPad_Phrases[ind]] for ind in 
                    range(0, len(ScratchPad_Times))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    #print(ScratchPad_New)
    return ScratchPad_New

In [11]:
def FixKeysSelected(KeysSelected_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    KeysSelected_New = list()
    
    # loop to combine phrases divided by commas
    KeysSelectedInd = -1 
    while KeysSelectedInd < len(KeysSelected_Old)-1:
        KeysSelectedInd = KeysSelectedInd + 1
        
        if KeysSelected_Old[KeysSelectedInd][1].count(',') > 0:
            
            keys_split = KeysSelected_Old[KeysSelectedInd][1].split("\r\n")
            del keys_split[0]
            del keys_split[-1]
            
            keys_split = [key.split(',') for key in keys_split]
            
            KeysSelected_New.extend(keys_split)
        else:
            KeysSelected_New.append(KeysSelected_Old[KeysSelectedInd])
        
    
    return KeysSelected_New

In [12]:
def ComputeDwellTime(userKeys, full_path):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    TimeDwellOrig = 800
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    if session_folder_name in dict_dwellTimeOrig_not800:
        TimeDwellOrig = dict_dwellTimeOrig_not800[session_folder_name]
    
    #print(TimeDwellOrig)
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        #print(key[1])
        if key[1] == 'IncreaseDwellTime':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseDwellTime':
            #print(key[2])
            if float(key[2]) == 1:
                # this is a quick-fix for something weird happening -- two consecutive 'DecreaseDwellTime' selected,
                # but the second one actually does not take effect on the keyboard, and needs to be not counted
                if session_folder_name == '2019-01-17-15-03-40_1' and nKey == 147:
                    continue
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [13]:
def stimPhrasesEdit(PhraseLog, full_path):
   
    # Now extract phrases from the phrase file
    phraseStim_Phrases = [item[1] for item in PhraseLog]
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
        
    phraseStim_PhrasesReduced, phraseStim_timeReduced = zip(*[(x[0], PhraseLog[phraseStim_Phrases.index(x[0])][0]) for x in groupby(phraseStim_Phrases)])
    
    PhraseLogReduced = [[phraseStim_timeReduced[i], phraseStim_PhrasesReduced[i]] for i in range(0, len(phraseStim_PhrasesReduced))]
    
    if PhraseLogReduced[-1][1] == 'THE EXPERIMENT IS NOW DONE':
        del PhraseLogReduced[-1]
        
    if PhraseLogReduced[0][1] == 'phraseText':
        del PhraseLogReduced[0]

    # Here, we want only the sentences typed
    notSentencesToType = list()
    for index in range(0,len(PhraseLogReduced)):
        sentence = PhraseLogReduced[index][1]
        if 'Svar på følgende spørgsmål' in sentence or 'Answer the question:' in sentence or 'What is the complete name of your university?' in sentence or '(give a score between 1 and 7)' in sentence or sentence == '':
            notSentencesToType.append(index)
         
    
    for index in sorted(notSentencesToType, reverse=True):
        del PhraseLogReduced[index]
    
    if session_folder_name in dict_phraseStim:
        #print('session in stim phrases found')
        index_to_be_removed = dict_phraseStim[session_folder_name]
    else:
        index_to_be_removed = []
        
    #print(index_to_be_removed)
    
    if index_to_be_removed:
        for index in sorted(index_to_be_removed, reverse=True):
            del PhraseLogReduced[index]
        
    return PhraseLogReduced

In [14]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [15]:
# function to convert list of date and time into datetime format list
def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [16]:
def OptiKeyTypingTime(UserKeys):
    
    TimeTyping = dict()
    
    time1, t1, t2 = UserKeys[0][0].partition('+')
    startTime = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    time2, t1, t2 = UserKeys[-1][0].partition('+')
    endTime = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    TimeTyping['startTime'] = startTime
    TimeTyping['endTime'] = endTime
    
    return TimeTyping

In [17]:
def timeTypingStart(userKeys):
    # From the user keys, find when the user actually starts typing, after having looked at the phrase and all the other 
    # function keys
    
    timeTypingStartInd = 0
    
    timeTypingStartIndList = list()
            
    timeUserTimeInd = 0
    
    ind = 0
    # Get start time of first trial
    
    while ind < len(userKeys):
        #print(len(userKeys[ind][1]))
        if len(userKeys[ind][1]) > 1:
            ind = ind + 1
        else:
            timeTypingStartInd = ind
            timeTypingStartIndList.append(ind)
            break
    
    #print(timeTypingStartInd)
    # Get every next phrase start timings
    while ind < len(userKeys):
        
        if userKeys[ind][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
            
            #timeTypingStartIndList.append(ind+1)
            for ind2 in range(ind+1, len(userKeys)):
                if len(userKeys[ind2][1]) > 1:
                    ind = ind + 1
                    continue
                elif userKeys[ind2][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
                    ind = ind + 1
                    continue
                else:
                    ind = ind2
                    timeTypingStartIndList.append(ind)
                    break
                    
        else:
            ind = ind + 1
            
    #print(timeTypingStartIndList)
    
    return timeTypingStartIndList

In [18]:
def findAndRemoveTrials(session_name, dictionary_saved, trials, replacingList):
    # function to check the session_name in the dictionary_saved and remove those trials from the dictionary_trial
    
    if session_name in dictionary_saved:
        index_list = dictionary_saved[session_name]
    else:
        index_list = replacingList
    
    
    if index_list:
        if type(trials) == list:
            for index in sorted(index_list, reverse=True):
                del trials[index]
                
        else:
            for index in sorted(index_list, reverse=True):
                del trials['start'][index]
                del trials['end'][index]
        
    return trials    

In [19]:
def scratchPadPhraseEdit(phraseUser, subjName, full_path, picture):
    phraseUserEnd = list()
    
    
    
    for row_ind in range(0, len(phraseUser)):
        if row_ind!= 0 and phraseUser[row_ind][1] == '':
            if phraseUser[row_ind-1][1] != 'scratchPadText':
                if len(phraseUser[row_ind-1][1])>2: # this also removes any answers on the difficulty of the sentence
                    phraseUserEnd.append(phraseUser[row_ind-1])
                    #print(phraseUser[row_ind-1])
                    
    #print(' (1) phrases reduced to :       ', phraseUserEnd)
    # remove first two trials of baseline question and text composition
    if 'not_described' in picture or '2019-01-14-14-58-30' in full_path or '2019-02-06-12-37-45_2' in full_path or '2019-02-18-10-28-35_2' in full_path: 
        # yss has not copied one of the sentences, aq_session1_2 has not written the baseline question, ls2_session4_2 did not describe the picture
        del phraseUserEnd[0]
    else:
        del phraseUserEnd[0:2]
      
    session_folder_name = full_path.split('\\')[-1]
    
    if session_folder_name in dict_phraseUser:
        index_to_be_removed = dict_phraseUser[session_folder_name]
        #print('session in user phrases found')
    else:
        index_to_be_removed = []
        
    
    if index_to_be_removed:
        for index in sorted(index_to_be_removed, reverse=True):
            del phraseUserEnd[index]
        
    
    #print(' (2) phrases reduced to:       ', phraseUserEnd)
    
    # remove the initial rating of complexity, if they have written it:
    for index in range(0,len(phraseUserEnd)):
        sentence = phraseUserEnd[index][1]
        if sentence[0].isdigit():
            
            # if there is also a space after the digit:
            
            if sentence[1] == ' ':
                phraseUserEnd[index][1] = phraseUserEnd[index][1][2:]
            else:
                phraseUserEnd[index][1] = phraseUserEnd[index][1][1:]
            
        elif sentence[1].isdigit():
            if sentence[2] == ' ':
                phraseUserEnd[index][1] = phraseUserEnd[index][1][3:]
            else:
                phraseUserEnd[index][1] = phraseUserEnd[index][1][2:]
    
    #print(' (3) phrases reduced to:       ', phraseUserEnd)
    
    return phraseUserEnd

In [20]:
def FindExptStartEndTimes(KeysSelected, timeTyping, full_path):
    # function to find start and end of tasks in experiments
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    timeTrialDict = dict()
    timeTrialDict = {'start': [],
                    'end':[]}
    
    nTrial = -1
    
    
    
    for keys in KeysSelected:
        
            
        
        if keys[1] == 'NextPhrase':
            nTrial = nTrial + 1
            time1, t1, t2 = keys[0].partition('+')
            endTimeTrial = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
            if nTrial != 0:
                # print('end: ', endTimeTrial)
                #print('')
                timeTrialDict['end'].append(endTimeTrial)
            
            
            startTimeTrial = endTimeTrial + datetime.timedelta(seconds=5)
            
            #print('start: ', startTimeTrial)
            timeTrialDict['start'].append(startTimeTrial)
        
    del timeTrialDict['start'][-1]
    
    
    # remove the extra selections of NewPhrase at the end of some sessions
    replacingList = []
    timeTrialDict = findAndRemoveTrials(session_folder_name, dict_keySelectionOfNextPhrase, timeTrialDict, replacingList)
    
    timeTrialDict_copy = copy.deepcopy(timeTrialDict)
    
    # separate the reading and writing trials for some participants who read in the actual trial, but write in the next
    # trial
    if session_folder_name in dict_keySelection_ReadingTrials:
        # check the reading and writing separate dictionaries
        print('reading and writing sessions are separate')
        
        #print(len(timeTrialDict['start']))
        # writing trials - 
        timeTrialDict_writing = findAndRemoveTrials(session_folder_name, dict_keySelection_WritingTrials, timeTrialDict, replacingList)
        #print(len(timeTrialDict_copy['start']))
        
        # reading trials
        timeTrialDict_reading = findAndRemoveTrials(session_folder_name, dict_keySelection_ReadingTrials, timeTrialDict_copy, replacingList)
    else:
        # some participants skip some sentences, and then it affects the scoreQuestions too. Remove the skipped sentences or 
        # remove the score questions 
        # for these participants, reading and writing trials are the same
        
        #print('same reading and writing trials')
        scoreQuestions = [0, 1, 3, 5, 7, 9, 11]
        timeTrialDict = findAndRemoveTrials(session_folder_name, dict_keySelectionNotCompleted, timeTrialDict, scoreQuestions)
        
        # most of the skipped sentences are removed, but for those that are not removed
        timeTrialDict_writing = findAndRemoveTrials(session_folder_name, dict_keySelection_phraseStim, timeTrialDict, replacingList)
        
        timeTrialDict_reading = timeTrialDict_writing 
        
    
          
    return timeTrialDict_reading, timeTrialDict_writing 

In [21]:
def FindReadingPartsOfTrial_inKeysSelected(EventTrials_reading, KeysSelected_new, full_path):
    # find the reading end in the pupil size data
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    KeysSelected_timeStr = [key[0] for key in KeysSelected_new]
    KeysSelected_time = timeConversion(KeysSelected_timeStr)
    
    KeysSelected_keys = [key[1] for key in KeysSelected_new]
    
    EventReading = dict()    
    EventReading['start'] = list()
    EventReading['end'] = list()
    
    EventReading_index = dict()    
    EventReading_index['start'] = list()
    EventReading_index['end'] = list()
    
    for ind, startTrialTime_afterCoolDown in enumerate(EventTrials_reading['start']):
        
        startTrialTime, startTrialInd = nearestTimePoint(KeysSelected_time, startTrialTime_afterCoolDown)
        
        EventReading['start'].append(startTrialTime)
        EventReading_index['start'].append(startTrialInd)
        
        endTrialTime = EventTrials_reading['end'][ind]
        endTrialInd = KeysSelected_time.index(endTrialTime)
        
        keysSelected_trial = KeysSelected_keys[startTrialInd:endTrialInd]
        
        # compute the end of reading time -- which is taken when the keyboard with phrases key is pressed
        if session_folder_name not in dict_keyboardNotChange_ReadingTrials:
            
            endReading_keyInd = startTrialInd + keysSelected_trial.index('KeyboardWithPhrases') - 1
            endReading_keyTime = KeysSelected_time[endReading_keyInd]
            
        else:
            if ind == dict_keyboardNotChange_ReadingTrials[session_folder_name]:
                endReading_keyInd = startTrialInd + keysSelected_trial.index('Sleep') - 1
                endReading_keyTime = KeysSelected_time[endReading_keyInd]
                
            else:
                endReading_keyInd = startTrialInd + keysSelected_trial.index('KeyboardWithPhrases') - 1
                endReading_keyTime = KeysSelected_time[endReading_keyInd]
                
                
        EventReading['end'].append(KeysSelected_time[endReading_keyInd+1])
        EventReading_index['end'].append(endReading_keyInd+1)
        
        #print(ind)
        #print('reading: ', EventReading['start'][ind], EventReading['end'][ind])
        
    
    
    return EventReading, EventReading_index

In [22]:
def FindWritingPartsOfTrial_inKeysSelected(EventTrials_writing, KeysSelected_new, EventReading):
    
    KeysSelected_timeStr = [key[0] for key in KeysSelected_new]
    KeysSelected_time = timeConversion(KeysSelected_timeStr)
    
    KeysSelected_keys = [key[1] for key in KeysSelected_new]
    
    EventWriting = dict()    
    EventWriting['start'] = list()
    EventWriting['end'] = list()
    
    EventWriting_index = dict()    
    EventWriting_index['start'] = list()
    EventWriting_index['end'] = list()
    
    for ind, startTrialTime_afterCoolDown in enumerate(EventTrials_writing['start']):
        
        startTrialTime, startTrialInd = nearestTimePoint(KeysSelected_time, startTrialTime_afterCoolDown)
        
        endTrialTime = EventTrials_writing['end'][ind]
        endTimeReading = EventReading['end'][ind]
        
        
        # for some participants, reading and writing trials are different. So their writing times will not be the end of 
        # the reading time.
        # Regardless, the writing time should start later than when the reading time ends.
        # We choose the starting time for writing as the one that is later than the start time from writing trials
        # and end time from reading trials
        
        if startTrialTime > endTimeReading:
            EventWriting['start'].append(startTrialTime)
            EventWriting_index['start'].append(startTrialInd)
        else:
            EventWriting['start'].append(endTimeReading)
            EventWriting_index['start'].append(KeysSelected_time.index(endTimeReading))
        
        EventWriting['end'].append(endTrialTime)
        EventWriting_index['end'].append(KeysSelected_time.index(endTrialTime))
        
        #print(ind)
        #print('writing: ', EventWriting['start'][ind], EventWriting['end'][ind])
        
    
    
    return EventWriting, EventWriting_index     

In [23]:
def CharactersTyped(EventWriting, KeysSelected_new):
    # total number of letters typed, 
    
    KeysSelected_timeStr = [key[0] for key in KeysSelected_new]
    KeysSelected_time = timeConversion(KeysSelected_timeStr)
    
    KeysSelected_keys = [key[1] for key in KeysSelected_new]
    
    nCharacters = list() 
    
    for i, timeStart in enumerate(EventWriting['start']):
        
        timeStartKey, timeStartKeyInd = nearestTimePoint(KeysSelected_time, timeStart)
        timeEndKey, timeEndKeyInd = nearestTimePoint(KeysSelected_time, EventWriting['end'][i])
        
        
        # remove keys that should not be counted towards the Characters - the score given before writing the sentence
        # till the characterKeyboard is selected, nothing to be selected till then
        nNotCharacters = 0
        
        for ind, key in enumerate(KeysSelected_keys[timeStartKeyInd-1:timeEndKeyInd+1]):
            if key == 'KeyboardWithPhrases':
                keyNewStart = ind+1
                break
            
        for ind, key in enumerate(KeysSelected_keys[timeStartKeyInd+keyNewStart-1:timeEndKeyInd+1]):
            if key == 'ScratchPad' or key == 'PhraseTextBlock' or key == 'BackSpace' or key == 'BackMany' or key == 'LeftShift' or key == 'Sleep' or 'Keyboard' in key:
                nNotCharacters = nNotCharacters + 1
        
        KeysSelected_trial_writingSentence = KeysSelected_keys[timeStartKeyInd+keyNewStart-1:timeEndKeyInd+1]
        KeysSelected_trial_time = KeysSelected_time[timeStartKeyInd+keyNewStart-1:timeEndKeyInd+1]
        
        #print(KeysSelected_trial_time[0], KeysSelected_trial_writingSentence[0])
        
        #print(KeysSelected_trial_time[-1], KeysSelected_trial_writingSentence[-1])
        
        #print('')
        
        
        nCharacters.append(len(KeysSelected_trial_writingSentence)-nNotCharacters)
        print(len(KeysSelected_trial_writingSentence)-nNotCharacters)
    
    return nCharacters

In [24]:
def EffectiveTimeFromUserKeys(UserKeys, Event, pathOfSession):
    
    UserKeysTimeStr = [val[0] for val in UserKeys]
    UserKeysTime = timeConversion(UserKeysTimeStr)
    
    timeActivation = 250
    
    # session name
    session_folder_name = pathOfSession.split('\\')[-1]
    
    timeTypingList = list()
    
    
    for indTrial, eventStart in enumerate(Event['start']):
        eventEnd = Event['end'][indTrial]
        eventStart_UserKeys, eventStartUserKeysInd = nearestTimePoint(UserKeysTime, eventStart)
        eventEnd_UserKeys, eventEnd_UserKeysInd = nearestTimePoint(UserKeysTime, eventEnd)
        time_trial = 0
        
        keySelectedPrevious = ''
        
        for indUserKey in range(eventStartUserKeysInd, eventEnd_UserKeysInd):
            
            if len(UserKeys[indUserKey][1]) == 1 or UserKeys[indUserKey][1] in list_keysToBeCounted or 'Suggestion' in UserKeys[indUserKey][1]:
                
                if len(UserKeys[indUserKey+1]) < 4 or float(UserKeys[indUserKey][3]) > float(UserKeys[indUserKey+1][3]) or UserKeys[indUserKey][2]==1:
                    
                    #print(UserKeys[indUserKey])
                    
                        
                    # Sometimes, in between selection of a letter, there is a small pause and activation of another letter,
                    # indicated by a '0' activation of the second letter during the first letter selection.
                    # This requires adding only the activation time for the second letter, not the dwell-time for the 
                    # current letter, which will be completed later on
                    
                    if len(UserKeys[indUserKey+1])>3 and len(UserKeys[indUserKey+2])>3:
                        if float(UserKeys[indUserKey][3]) > float(UserKeys[indUserKey+1][3]):
                            if float(UserKeys[indUserKey+1][3]) == 0:
                                if UserKeys[indUserKey+2][1] == UserKeys[indUserKey][1]:
                                    time_trial = time_trial + timeActivation
                                    #print('Only timeActivation added')
                                    if UserKeys[indUserKey][2]==1:
                                        keySelectedPrevious = UserKeys[indUserKey][1]
                                    continue
                    
                    if keySelectedPrevious == UserKeys[indUserKey][1]: # if the key selection is accidentally continued,
                        # activation time should not be counted
                        time_trial = time_trial + float(UserKeys[indUserKey][3])
                        #print('Current dwell-time added', keySelectedPrevious)
                        
                    else:
                        #print('timeActivation added')
                        time_trial = time_trial + float(UserKeys[indUserKey][3]) + timeActivation
                    
                    if UserKeys[indUserKey][2]==1:
                            keySelectedPrevious = UserKeys[indUserKey][1]
        
        print(time_trial/1000)                    
        timeTypingList.append(time_trial/1000) # add the time in seconds
        
    
    return timeTypingList

In [29]:
def LingeringOnKeys(userKeys):
    
    timeLingeringOnKeysList = list()
    
    keyPrevious = userKeys[0]
    
    keySelected = 'Sleep'
    
    for i, userKey in enumerate(userKeys):
        
        if float(userKey[2]) == 1:
            keySelected = userKey[1]
            keyPrevious = userKey
            
            continue
        
        if keySelected != userKey[1]:
            keyPrevious = userKey
            continue
        
        if keyPrevious[1] != userKey[1]:
            keyPrevious = userKey
            continue
        
        # Proceeding here means keyPrevious and keyCurrent are the same
        if len(keyPrevious[1]) > 1: # it is not a letter
            keyPrevious = userKey
            continue
            
        if len(userKey[1]) > 1: # it is not a letter
            keyPrevious = userKey
            continue
        
        time1, t1, t2 = userKey[0].partition('+')
        timeCurrent = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
        
        keyPrevious = userKey
        
        if i <= len(userKeys) - 1:
            if userKey[1] == userKeys[i+1][1]:
                if float(userKeys[i+1][2]) > 0:
                    continue
                else:
                    timeLingeringOnKeysList.append([timeCurrent, float(userKey[3])])
                    #print(userKey)
                    keySelected = 'Sleep'
            else:
                if float(userKey[2]) == 0:
                    timeLingeringOnKeysList.append([timeCurrent, 250])
                    #print('adding 250', userKey)
                else:
                    timeLingeringOnKeysList.append([timeCurrent, float(userKey[3])])
                    #print(userKey)
                keySelected = 'Sleep'
    
        
    
    return timeLingeringOnKeysList  

In [30]:
class DataForEverySubject:
    
    timeStamp = ''
    timeLingeringOnKeys = ''
    
    resultPathName = ''
   
    
    def printInfo(self):
        dataFrame = pd.DataFrame(list(zip(self.timeStamp, self.timeLingeringOnKeys)), columns=['timeStamp', 'timeLingeringOnKeys'])
        
        return dataFrame
    
    def AddToFile(self):
        
        dataFrame = pd.DataFrame(list(zip(self.timeStamp, self.timeLingeringOnKeys)), columns=['timeStamp', 'timeLingeringOnKeys'])
        
        #workbook = xlsxwriter.Workbook(self.resultPathName)
        #book = workbook.add_worksheet()
        
        #book = load_workbook(self.resultPathName)
        writer = pd.ExcelWriter(self.resultPathName, engine='xlsxwriter')
        #writer.book = book
        #writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
        
        dataFrame.to_excel(writer, index = False, header = False)
        workbook  = writer.book
        worksheet = writer.sheets['Sheet1']
        writer.save()
    
    

In [32]:

dataFolderName = r'E:\Data\Data' # accessing external hard disk with the data
a = re.compile('(?<=Data\\\\Data\\\\)(.*)(?=\\\\[1-9])')
subjectName_listElement = 3

#dataFolderName = r'C:\DTU\Data\201901_JanuaryExpt' # accessing data saved in the computer
#a = re.compile('(?<=Data\\\\201901_JanuaryExpt\\\\)(.*)(?=\\\\[1-9])')
#subjectName_listElement = 4



for root, dirs, subfolder in os.walk(dataFolderName):
    
    technique = 'dwell_time'
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'noData' in root or 'Trial' in root or 'trial' in root or 'Nothing' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'Jonas' in root or 'Praktikant' in root or 'Villads' in root:
            continue
            
        #if 'af\\1' not in root:
        #    continue
        if 'Picture' in root:
            continue
        #if '_MS' not in root:
        #    continue
        
        
            
        keysSelected = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerScratchPad = csv.reader(fScratchPad, quotechar=None)
                    scratchPad = list(readerScratchPad)
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the scratchpad log file')
            
            if fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
                        
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8',  newline='')
                    readerUserKey = csv.reader(fUserKey, quotechar=None)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog, quotechar=None)
                    phraseLog = list(readerPhraseLog)
                    
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                       
            if fnmatch.fnmatch(file, 'multiKey*'):
                technique = 'multiKey_selection'
            
            if fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerGazeLog = csv.reader(fGazeLog, quotechar=None)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0])
                    gazeLog.remove(gazeLog[-1])
                    
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the scratchpad log file')
            
                    
                     
        if keysSelected is None or userKeys is None or phraseLog is None or gazeLog is None or scratchPad is None:
            continue
        else:
                
            print('subject path', root)
            subjAndSessionName = '__'.join(root.split('\\')[subjectName_listElement:])
            subjName = subjAndSessionName.split('__')[0]
            print('subject and session name: ', subjAndSessionName)
            sessionFolderName = root.split('\\')[-1]
            
            resultFileName = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\Subjects\Performance\TimeLingeringOnKeys' \
            + '\\' + subjName + '\\' + subjAndSessionName +  '.xlsx'
            
            # fix scratchpad due to comma related file changes
            scratchPad_new = FixScratchPad(scratchPad)
            
            # fix phraselog due to comma related file changes
            phraseLog_new = FixScratchPad(phraseLog)
            
            # fix userKeys due to comma related file changes
            userKeys_new = FixUserKeys(userKeys)
            
            # find dwell time of typing
            userKeys_wDwellTime = ComputeDwellTime(userKeys_new, root)
            
            
            # need to fix keys selected, where rows get combined because of an inverted comma
            keysSelected_new = FixKeysSelected(keysSelected)
            
            # find time between selection of 1 key and start of selection of the next
            timeLingeringOnKeysList = LingeringOnKeys(userKeys_wDwellTime)
            
            
            timeList = [key[0] for key in timeLingeringOnKeysList]
            timeLingeringOnKeys = [key[1] for key in timeLingeringOnKeysList]
            
            if '1stPart' in root:
                print('1stPart')
                timeList1 = timeList
                timeLingeringOnKeys1 = timeLingeringOnKeys
                continue
            
            
            
            if '2ndPart' in root:
                print('2ndPart')
                timeList2 = timeList
                timeLingeringOnKeys2 = timeLingeringOnKeys
                
                timeList = timeList1 + timeList2
                timeLingeringOnKeys = timeLingeringOnKeys1 + timeLingeringOnKeys2
                
                timeList1 = list()
                timeLingeringOnKeys1 = list()
                
                
            dataToSave = DataForEverySubject()
            dataToSave.timeStamp = timeList
            dataToSave.timeLingeringOnKeys = timeLingeringOnKeys
            dataToSave.resultPathName = resultFileName
            
            print(dataToSave.printInfo())
            
            
            dataToSave.AddToFile()
            
            
            
            
            

subject path E:\Data\Data\ac\1\2019-02-11-11-18-30_1
subject and session name:  ac__1__2019-02-11-11-18-30_1
                    timeStamp  timeLingeringOnKeys
0  2019-02-11 11:18:38.091839              32.6884
1  2019-02-11 11:19:35.254736              11.0010
2  2019-02-11 11:19:41.827198              32.9137
3  2019-02-11 11:20:10.764611              32.9130
4  2019-02-11 11:20:33.752175              33.9178
5  2019-02-11 11:20:52.200404              44.8773
6  2019-02-11 11:21:33.207252             188.2093
7  2019-02-11 11:21:38.293028              77.5148
8  2019-02-11 11:21:49.508539              67.1298
9  2019-02-11 11:22:41.674187              33.3388
10 2019-02-11 11:23:08.070849             221.5287
11 2019-02-11 11:26:40.110688              11.9677
12 2019-02-11 11:26:56.110679              66.8831
13 2019-02-11 11:27:20.182411              32.6712
14 2019-02-11 11:27:24.578755               8.9765
15 2019-02-11 11:27:59.998397             288.1439
16 2019-02-11 11:30:00.5

subject path E:\Data\Data\ac\4\2019-02-18-14-59-21_1
subject and session name:  ac__4__2019-02-18-14-59-21_1
                    timeStamp  timeLingeringOnKeys
0  2019-02-18 15:07:40.699844              10.9704
1  2019-02-18 15:08:02.469057              10.9744
2  2019-02-18 15:09:33.299007             233.3280
3  2019-02-18 15:11:37.612163              22.6927
4  2019-02-18 15:11:42.230313              32.9122
5  2019-02-18 15:12:14.833481             121.4154
6  2019-02-18 15:15:49.947261              78.7896
7  2019-02-18 15:20:06.932737             176.6549
8  2019-02-18 15:20:24.313263             122.9807
9  2019-02-18 15:21:08.369684              43.7909
10 2019-02-18 15:21:59.367373              22.9385
subject path E:\Data\Data\ac\4\2019-2-18-15-23-54_2
subject and session name:  ac__4__2019-2-18-15-23-54_2
                   timeStamp  timeLingeringOnKeys
0 2019-02-18 15:26:32.273727              43.8825
1 2019-02-18 15:26:51.416668              33.4437
2 2019-02-18 15:27:32.

subject path E:\Data\Data\af\2\2019-01-17-15-03-40_1
subject and session name:  af__2__2019-01-17-15-03-40_1
                   timeStamp  timeLingeringOnKeys
0 2019-01-17 15:03:54.638568              32.9122
1 2019-01-17 15:10:54.337287             250.0000
2 2019-01-17 15:14:21.828280             250.0000
3 2019-01-17 15:16:39.434464             250.0000
4 2019-01-17 15:19:46.777298             250.0000
5 2019-01-17 15:23:24.292609             250.0000
6 2019-01-17 15:24:15.538994             250.0000
subject path E:\Data\Data\af\2\2019-01-17-15-27-20_1stPart_2
subject and session name:  af__2__2019-01-17-15-27-20_1stPart_2
1stPart
subject path E:\Data\Data\af\2\2019-01-17-16-03-27_2ndPart_2
subject and session name:  af__2__2019-01-17-16-03-27_2ndPart_2
2ndPart
                   timeStamp  timeLingeringOnKeys
0 2019-01-17 15:32:11.993186             250.0000
1 2019-01-17 15:32:15.569532             250.0000
2 2019-01-17 15:32:30.485002             250.0000
3 2019-01-17 15:34:22.475

subject path E:\Data\Data\af\4\2019-02-12-11-21-21_2
subject and session name:  af__4__2019-02-12-11-21-21_2
                   timeStamp  timeLingeringOnKeys
0 2019-02-12 11:21:38.183398              88.9507
1 2019-02-12 11:21:47.177689              12.3384
subject path E:\Data\Data\af\5\2019-02-27-15-08-32_1
subject and session name:  af__5__2019-02-27-15-08-32_1
                    timeStamp  timeLingeringOnKeys
0  2019-02-27 15:09:11.987726              22.9392
1  2019-02-27 15:09:40.834261              89.6784
2  2019-02-27 15:10:27.615006              44.9041
3  2019-02-27 15:10:30.701373              33.3534
4  2019-02-27 15:10:46.779148              33.4814
5  2019-02-27 15:11:57.065351              89.5716
6  2019-02-27 15:11:59.163704              43.6911
7  2019-02-27 15:12:51.594607              90.0944
8  2019-02-27 15:12:55.468536              21.1929
9  2019-02-27 15:13:10.537994              54.4510
10 2019-02-27 15:13:19.764726              77.9648
11 2019-02-27 15:13:

subject path E:\Data\Data\aq\1\2019-02-06-12-37-45_2
subject and session name:  aq__1__2019-02-06-12-37-45_2
                    timeStamp  timeLingeringOnKeys
0  2019-02-06 12:38:47.453612              21.7497
1  2019-02-06 12:39:34.098663              21.9407
2  2019-02-06 12:40:10.771199              53.8805
3  2019-02-06 12:40:24.229448              44.3581
4  2019-02-06 12:40:48.733717             143.3995
5  2019-02-06 12:40:59.437901              21.9990
6  2019-02-06 12:42:09.775580              55.0731
7  2019-02-06 12:42:32.512223             250.0000
8  2019-02-06 12:42:48.903412              56.7286
9  2019-02-06 12:43:54.172820              21.9479
10 2019-02-06 12:44:01.723940              22.9382
11 2019-02-06 12:44:05.998195              76.6489
12 2019-02-06 12:44:07.653490              55.8526
13 2019-02-06 12:44:09.162620             132.8422
14 2019-02-06 12:44:17.001192              44.9379
15 2019-02-06 12:44:37.407985             641.4960
16 2019-02-06 12:46:10.9

...

subject path E:\Data\Data\ph\3\2019-02-07-16-11-24_1
subject and session name:  ph__3__2019-02-07-16-11-24_1
                    timeStamp  timeLingeringOnKeys
0  2019-02-07 16:12:09.925014              77.8090
1  2019-02-07 16:13:22.431146              34.7731
2  2019-02-07 16:14:47.225488             322.0261
3  2019-02-07 16:14:58.518790              43.8834
4  2019-02-07 16:14:59.960808              55.1470
5  2019-02-07 16:15:52.712838              33.2331
6  2019-02-07 16:16:04.904767              33.9146
7  2019-02-07 16:16:25.323512             189.1115
8  2019-02-07 16:16:38.681157              65.9164
9  2019-02-07 16:17:03.652058              45.6696
10 2019-02-07 16:17:35.303824             250.0000
11 2019-02-07 16:17:41.598266             250.0000
12 2019-02-07 16:17:56.468296             250.0000
13 2019-02-07 16:18:04.675734              44.9119
14 2019-02-07 16:19:21.227181              11.9564
15 2019-02-07 16:19:29.166069              33.2726
16 2019-02-07 16:19:40.3

subject path E:\Data\Data\rh\1\2019-03-07-16-17-30_1
subject and session name:  rh__1__2019-03-07-16-17-30_1
                    timeStamp  timeLingeringOnKeys
0  2019-03-07 16:18:09.485922             277.7703
1  2019-03-07 16:19:56.184490              77.5583
2  2019-03-07 16:20:03.812063             321.3663
3  2019-03-07 16:20:38.689117              32.6887
4  2019-03-07 16:21:34.260529              43.9704
5  2019-03-07 16:21:47.296666             278.4952
6  2019-03-07 16:22:12.440542              44.2775
7  2019-03-07 16:22:49.536265              66.2081
8  2019-03-07 16:23:28.603509              99.0820
9  2019-03-07 16:24:23.694439              54.8525
10 2019-03-07 16:25:35.011665             289.1778
11 2019-03-07 16:25:36.696692               9.8243
12 2019-03-07 16:26:32.859844              43.0349
13 2019-03-07 16:26:51.603846              12.2431
14 2019-03-07 16:27:20.224465             110.9526
15 2019-03-07 16:27:30.107261              11.9675
16 2019-03-07 16:27:43.2

subject path E:\Data\Data\rh\5\2019-04-15-14-49-27_2
subject and session name:  rh__5__2019-04-15-14-49-27_2
                    timeStamp  timeLingeringOnKeys
0  2019-04-15 14:49:55.229068              32.3496
1  2019-04-15 14:51:58.614401              89.6825
2  2019-04-15 14:52:00.147544              33.0920
3  2019-04-15 14:57:28.456017             788.9955
4  2019-04-15 14:57:29.487383              63.7984
5  2019-04-15 15:00:31.569089              43.9341
6  2019-04-15 15:02:56.004647              66.5422
7  2019-04-15 15:03:30.991165             731.9675
8  2019-04-15 15:04:24.143404              78.0557
9  2019-04-15 15:07:38.666006             331.2371
10 2019-04-15 15:14:23.216953              89.4696
11 2019-04-15 15:15:08.174382             110.8087
12 2019-04-15 15:16:14.030132              22.0325
13 2019-04-15 15:21:52.090771              33.0979
14 2019-04-15 15:22:13.408183              76.6155
15 2019-04-15 15:22:21.816265              55.8809
16 2019-04-15 15:22:28.7

subject path E:\Data\Data\ys\2\2019-01-16-15-59-55_2
subject and session name:  ys__2__2019-01-16-15-59-55_2
                    timeStamp  timeLingeringOnKeys
0  2019-01-16 16:02:58.057119              10.9710
1  2019-01-16 16:03:00.031970              77.3024
2  2019-01-16 16:03:47.165897              42.8829
3  2019-01-16 16:03:52.018513              23.4451
4  2019-01-16 16:04:16.267805              32.9108
5  2019-01-16 16:05:27.386729              33.9094
6  2019-01-16 16:05:34.870710              21.9428
7  2019-01-16 16:05:36.946495              22.9394
8  2019-01-16 16:06:57.332689              55.8565
9  2019-01-16 16:08:58.280695              32.9115
10 2019-01-16 16:09:03.453031              21.9417
11 2019-01-16 16:11:04.906998              44.8793
12 2019-01-16 16:11:16.520090              39.8930
13 2019-01-16 16:11:23.815225              77.7921
14 2019-01-16 16:11:45.122223              42.4084
subject path E:\Data\Data\ys\3\2019-01-17-15-05-1_1
subject and session nam

subject path E:\Data\Data\ys\4_MS\2019-01-30-11-57-3_2
subject and session name:  ys__4_MS__2019-01-30-11-57-3_2
                    timeStamp  timeLingeringOnKeys
0  2019-01-30 11:57:42.703951              23.0833
1  2019-01-30 12:00:06.975681               9.9725
2  2019-01-30 12:00:09.550910             198.1449
3  2019-01-30 12:01:19.013133             389.5715
4  2019-01-30 12:01:35.301827              32.9174
5  2019-01-30 12:03:00.041425              22.9351
6  2019-01-30 12:03:06.380495             321.6862
7  2019-01-30 12:03:12.221419             299.7260
8  2019-01-30 12:03:13.998006              22.7015
9  2019-01-30 12:03:27.955552              11.2365
10 2019-01-30 12:03:37.136649             288.1067
11 2019-01-30 12:04:08.766449             399.3845
12 2019-01-30 12:04:16.702000             250.0000
13 2019-01-30 12:05:24.478762             200.1564
14 2019-01-30 12:06:32.579515             398.6124
15 2019-01-30 12:08:19.026325              33.8894
16 2019-01-30 12:08: